In [23]:
import itertools
from opencage.geocoder import OpenCageGeocode
from bs4 import BeautifulSoup as bs
import numpy as np
from selenium import webdriver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import os
import time
import requests
import warnings
warnings.filterwarnings('ignore')

In [24]:
path = "info/Aviation_Accident_Report_mine.csv"
aar_csv = pd.read_csv(path)
aar_csv

,Unnamed: 0,Report Number,NTSB Title,Accident Date,Report Date,City,State
0,0,ASR-20-06,Provide Inspectors with Automatic Notification...,6/21/2019,12/16/2020,Mokuleia,HI
1,1,ASR-20-05,F&M Enterprises and Stratus Oil Filter Adapter...,11/30/2020,11/30/2020,Multiple,Multiple
2,2,AAR-20-02,Rapid Descent and Crash into Water Atlas Air I...,2/23/2019,7/14/2020,Trinity Bay,TX
3,3,ASR-20-04,"Install Flight Data, Audio, and Image Recorder...",7/1/2017,5/19/2020,Multiple,Multiple
4,4,AAR-20-01,Helicopter Air Ambulance Collision with Terrai...,1/29/2019,5/19/2020,Zaleski,OH
...,...,...,...,...,...,...,...
475,514,AAR-67-AE,"​Rexall Drug and Chemical Company Lear Jet 23,...",4/12/1966,12/19/1967,Clarendon,TX
476,516,AAR-67-AB,"​Lear Jet Corporation, Lear Jet Model 23, N8041J",10/21/1965,12/11/1967,Jackson,MI
477,517,AAR-67-AF,"​West Coast Airlines, Inc., DC-9, N9101",10/1/1966,12/11/1967,Wemme,OR
478,518,AAR-67-AD,"​Japan Air Lines, Ltd., Douglas DC-8-33, JA-8006",12/25/1965,6/26/1967,San Francisco,CA


In [26]:
cities = aar_csv["City"].to_list()
states = aar_csv["State"].to_list()

key = ""

lati = []
longi = []


for (city, state) in zip(cities, states):
    try:
        geocoder = OpenCageGeocode(key)
        query = f'{city}, {state}'
        results = geocoder.geocode(query)
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
        lati.append(lat)
        longi.append(lng)
        time.sleep(1)
    except KeyError:
        print(city)


In [28]:

aar_csv['Latitude'] = lati
aar_csv['Longitude'] = longi

In [30]:
aar_csv
aar_csv.to_csv('info/aviation_and_coordinates.csv', index = False)


In [41]:
#Just testing stuff.... this is probably useless so ignore it please. Or don't. It's up to you really.

url ='https://aviation-safety.net/database/dblist.php?Year=1971'
#soup = BeautifulSoup(requests.get(the_url).text)
#aar_tables = pd.read_html(str(soup.find_all('table')[9]))
r = requests.get(url)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(r)

AttributeError: 'function' object has no attribute 'PrettyPrinter'

In [51]:


driver = webdriver.Chrome()
driver.get("https://aviation-safety.net/database/dblist.php?Year=1971")

html = driver.page_source

tables = pd.read_html(html)

driver.close()



AttributeError: 'list' object has no attribute 'head'

In [53]:
# THE DEED IS DONE AND I GOT THE TABLE

deaths = tables[0]
deaths

,date,type,registration,operator,fat.,location,Unnamed: 6,pic,cat
0,02-JAN-1971,DH-106 Comet 4C,SU-ALC,UAA,16.0,near Tripoli Inte...,NaN,NaN,A1
1,03-JAN-1971,DC-8,NaN,National Airlines,0.0,Havana-José ...,NaN,NaN,H2
2,04-JAN-1971,Douglas C-47A (DC-3),N7,FAA,0.0,New York-La ...,NaN,NaN,A1
3,04-JAN-1971,SE-210 Caravelle III,F-BNKI,Air Inter,0.0,Paris-Orly A...,NaN,NaN,O1
4,07-JAN-1971,Antonov An-2M,VT-DZB,Khemka Aviation,0.0,near Patna,NaN,NaN,A1
...,...,...,...,...,...,...,...,...,...
95,24-MAY-1971,Grumman SA-16A Albatross (HU-16),AD.1-3,Spanish AF,0.0,near Braganza,NaN,NaN,A1
96,25-MAY-1971,Antonov An-12B,CCCP-11024,Aeroflot / Yakutsk,0.0,Batagay Airport,NaN,NaN,A1
97,25-MAY-1971,Curtiss C-46F,CC-CAZ,ALFA,4.0,near Santiago-Los...,NaN,NaN,A1
98,26-MAY-1971,Hawker Siddeley HS-125-400B,01,South African AF,3.0,Devil's Peak,NaN,NaN,A1


In [30]:
#Run this to get the code for 12 year airport data

key = ""

#Url params
pages = [1, 2, 3]
years = np.arange(2009, 2022, 1).tolist()

#List results
Dates = []
Type = []
Registration = []
Operator = []
Fatalities = []
Location = []
Lat2 = []
Lon2 = []
failed_location = []



for year in years:
    for page in pages:
        try:
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(f"https://aviation-safety.net/database/dblist.php?Year={year}&lang=&page={page}")
            html = driver.page_source
            a_tables = pd.read_html(html)
            accidents = a_tables[0]
            Location.append(accidents["location"])
            Dates.append(accidents["date"])
            Type.append(accidents["type"])
            Registration.append(accidents["registration"])
            Operator.append(accidents["operator"])
            Fatalities.append(accidents["fat."])
            
            driver.close()
        
        except (KeyError, ValueError, IndexError):
            failed_location.append(location)
            
            driver.close()
        
        





[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


...

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/tylerbrown/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


KeyboardInterrupt: 

In [29]:
len(Dates)

0

In [3]:

new_Dates = list(itertools.chain.from_iterable(Dates))
new_Type = list(itertools.chain.from_iterable(Type))
new_Registration = list(itertools.chain.from_iterable(Registration))
new_Operator = list(itertools.chain.from_iterable(Dates))
new_Fatalities = list(itertools.chain.from_iterable(Fatalities))
new_Location = list(itertools.chain.from_iterable(Location))
new_Cat = list(itertools.chain.from_iterable(Cat))


In [5]:
#12 year df

errors = {'Date': new_Dates,
        'Type': new_Type,
        'Registration': new_Registration,
        'Operator': new_Operator,
        'Fatalities': new_Fatalities,
        'Location': new_Location,
        'Cat': new_Cat
        }

twelve_df = pd.DataFrame(errors, columns = ['Date', 'Type', 'Registration',
                                            'Operator', 'Fatalities', 'Location', 'Cat'])

twelve_df


,Date,Type,Registration,Operator,Fatalities,Location,Cat
0,02-JAN-2009,Antonov An-12BP,S9-SAM,02-JAN-2009,0,Sharjah Airp...,A1
1,03-JAN-2009,Learjet 45XR,N279AJ,03-JAN-2009,0,Telluride Re...,A1
2,04-JAN-2009,Cessna 650 Citation VII,XA-PYN,04-JAN-2009,0,Vancouver In...,A2
3,04-JAN-2009,Cessna 550 Citation II,N815MA,04-JAN-2009,0,Wilmington-N...,A1
4,04-JAN-2009,Basler Turbo 67 (DC-3T),C-GEAJ,04-JAN-2009,0,Tony Camp,A2
...,...,...,...,...,...,...,...
2415,02-JAN-2021,Beech 200 Super King Air,N831WP,02-JAN-2021,0,White Plains...,A2
2416,09-JAN-2021,Boeing 737-524 (WL),PK-CLC,09-JAN-2021,62,near Jakarta-Soek...,A1
2417,09-JAN-2021,Cessna 560 Citation V,N3RB,09-JAN-2021,2,"near Pine Grove, OR",A1
2418,12-JAN-2021,Harbin Y-12-II,NaN,12-JAN-2021,4,near Voi,A1


In [6]:
twelve_df.to_csv('12_year_accident.csv', index = False)

In [69]:
#50 year df
mistakes = {'Date': new_Dates,
        'Type': new_Type,
        'Registration': new_Registration,
        'Operator': new_Operator,
        'Fatalities': new_Fatalities,
        'Location': new_Location,
        'Cat': new_Cat
        }

Accident_df = pd.DataFrame(mistakes, columns = ['Date', 'Type', 'Registration',
                                               'Operator', 'Fatalities', 'Location', 'Cat'])

Accident_df

,Date,Type,Registration,Operator,Fatalities,Location,Cat
0,02-JAN-1971,DH-106 Comet 4C,SU-ALC,02-JAN-1971,16,near Tripoli Inte...,A1
1,03-JAN-1971,DC-8,NaN,03-JAN-1971,0,Havana-José ...,H2
2,04-JAN-1971,Douglas C-47A (DC-3),N7,04-JAN-1971,0,New York-La ...,A1
3,04-JAN-1971,SE-210 Caravelle III,F-BNKI,04-JAN-1971,0,Paris-Orly A...,O1
4,07-JAN-1971,Antonov An-2M,VT-DZB,07-JAN-1971,0,near Patna,A1
...,...,...,...,...,...,...,...
11627,02-JAN-2021,Beech 200 Super King Air,N831WP,02-JAN-2021,0,White Plains...,A2
11628,09-JAN-2021,Boeing 737-524 (WL),PK-CLC,09-JAN-2021,62,near Jakarta-Soek...,A1
11629,09-JAN-2021,Cessna 560 Citation V,N3RB,09-JAN-2021,2,"near Pine Grove, OR",A1
11630,12-JAN-2021,Harbin Y-12-II,NaN,12-JAN-2021,4,near Voi,A1


In [70]:
Accident_df.to_csv('50_year_accident.csv', index = False)

In [8]:
#Beginning of 12 year coordinate work.

path2 = "12_year_accident.csv"
twelve_csv = pd.read_csv(path2)
twelve_csv

,Date,Type,Registration,Operator,Fatalities,Location,Cat
0,02-JAN-2009,Antonov An-12BP,S9-SAM,02-JAN-2009,0,Sharjah Airp...,A1
1,03-JAN-2009,Learjet 45XR,N279AJ,03-JAN-2009,0,Telluride Re...,A1
2,04-JAN-2009,Cessna 650 Citation VII,XA-PYN,04-JAN-2009,0,Vancouver In...,A2
3,04-JAN-2009,Cessna 550 Citation II,N815MA,04-JAN-2009,0,Wilmington-N...,A1
4,04-JAN-2009,Basler Turbo 67 (DC-3T),C-GEAJ,04-JAN-2009,0,Tony Camp,A2
...,...,...,...,...,...,...,...
2415,02-JAN-2021,Beech 200 Super King Air,N831WP,02-JAN-2021,0,White Plains...,A2
2416,09-JAN-2021,Boeing 737-524 (WL),PK-CLC,09-JAN-2021,62,near Jakarta-Soek...,A1
2417,09-JAN-2021,Cessna 560 Citation V,N3RB,09-JAN-2021,2,"near Pine Grove, OR",A1
2418,12-JAN-2021,Harbin Y-12-II,NaN,12-JAN-2021,4,near Voi,A1


In [18]:
#This is to test if the API will work with these vague locations..... and it does. What the heck?

key2 = "2808685076994e07aa9ee8b6023dfe44"

geocoder = OpenCageGeocode(key2)

query = 'Sharjah Airp...'  

results2 = geocoder.geocode(query)

results2

lato = results2[0]['geometry']['lat']

lngo = results2[0]['geometry']['lng']

print (lato, lngo)

[]

In [34]:
#12 year coordinate work.



key = "0238264761764379bba622428ace0518"

#Knowns
locations = twelve_csv["Location"].to_list()
dates = twelve_csv["Date"].to_list()
versions = twelve_csv["Type"].to_list()
regis = twelve_csv["Registration"].to_list()
operators = twelve_csv["Operator"].to_list()
fatalities = twelve_csv["Fatalities"].to_list()
locales = twelve_csv["Location"].to_list()


#Unknowns
place = []
day = []
version2 = []
regi_num = []
op = []
fatal = []
local = []
lats = []
longs = []


failed = []


for (location, date, version, regi, operator, fatality, locale) in zip(locations, dates, versions, regis, operators, fatalities, locales):
    try:
        geocoder = OpenCageGeocode(key)
        query = f'{location}'
        results = geocoder.geocode(query)
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
        
        place.append(location)
        day.append(date)
        version2.append(version)
        regi_num.append(regi)
        op.append(operator)
        fatal.append(fatality)
        local.append(locale)
        lats.append(lat)
        longs.append(lng)
        
    except (KeyError, IndexError):
        failed.append(location)



In [37]:
the_twelve = {'Date': day,
        'Type': version2,
        'Registration': regi_num,
        'Operator': op,
        'Fatalities': fatal,
        'Location': local,
        'Latitude': lats,
        'Longitude': longs
        }

final_twelve = pd.DataFrame(the_twelve, columns = ['Date', 'Type', 'Registration',
                                               'Operator', 'Fatalities', 'Location', 'Latitude', 'Longitude'])

final_twelve.to_csv('12_year_accident.csv', index = False)
final_twelve

,Date,Type,Registration,Operator,Fatalities,Location,Latitude,Longitude
0,03-JAN-2009,Learjet 45XR,N279AJ,03-JAN-2009,0,Telluride Re...,-21.100000,55.600000
1,04-JAN-2009,Cessna 650 Citation VII,XA-PYN,04-JAN-2009,0,Vancouver In...,49.297470,-123.026407
2,04-JAN-2009,Cessna 550 Citation II,N815MA,04-JAN-2009,0,Wilmington-N...,39.745947,-75.546589
3,04-JAN-2009,Basler Turbo 67 (DC-3T),C-GEAJ,04-JAN-2009,0,Tony Camp,44.883320,-123.998057
4,09-JAN-2009,Cessna 208 Caravan I,EP-858,09-JAN-2009,0,near Intuto,-3.490097,-74.781119
...,...,...,...,...,...,...,...,...
1561,30-DEC-2020,Cessna 208B Supervan 900,PZ-TSK,30-DEC-2020,0.0,Kawemhaken-L...,3.411489,-54.025328
1562,02-JAN-2021,Learjet 31A,PP-BBV,02-JAN-2021,0,Diamantina A...,-18.244086,-43.600649
1563,02-JAN-2021,Beech 200 Super King Air,N831WP,02-JAN-2021,0,White Plains...,41.033986,-73.762910
1564,09-JAN-2021,Cessna 560 Citation V,N3RB,09-JAN-2021,2,"near Pine Grove, OR",45.108728,-121.363676
